In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### ARIK ERTUGRUL ###
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
   cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
   print('Found existing cluster, use it.')
except ComputeTargetException:
   compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                          max_nodes=4)
   cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
### ARIK ERTUGRUL ###
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters
ps = RandomParameterSampling( {
        "learning_rate": normal(10, 3),
        "keep_probability": uniform(0.05, 0.1),
        "batch_size": choice(16, 32, 64, 128)
    }
)

# Specify a Policy
### ARIK ERTUGRUL ###
# Bayesian sampling does not support early termination. When using Bayesian sampling, set early_termination_policy = None
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)
# the early termination policy is applied at every interval when metrics are reported, starting at evaluation interval 5. 
# Any run whose best metric is less than (1/(1+0.1) or 91% of the best performing run will be terminated.

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
### ARIK ERTUGRUL ###
script_params = {
            '--C': 1.0,
            '--max_iter': 100,
        }
est = SKLearn(source_directory="./", 
                            script_params=script_params,
                            compute_target=cpu_cluster_name,
                            entry_script='train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
### ARIK ERTUGRUL ###
# https://docs.microsoft.com/fr-fr/python/api/azureml-train-core/azureml.train.hyperdrive.hyperdriveconfig?view=azure-ml-py
hyperdrive_config = HyperDriveConfig(estimator=est,
                                         hyperparameter_sampling=param_sampling, 
                                         policy=policy,
                                         primary_metric_name='Accuracy',
                                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                         max_total_runs=12,
                                         max_concurrent_runs=4)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### ARIK ERTUGRUL ###
hyperdrive_run = exp.submit(hyperdrive_config)

from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
import joblib
# Get your best run and save the model from that run.
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-train-scikit-learn

### ARIK ERTUGRUL ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print('\n learning rate:',parameter_values[3])
print('\n keep probability:',parameter_values[5])
print('\n batch size:',parameter_values[7])

model = best_run.register_model(model_name = 'bestmodel_HyperDrive', model_path = 'bestmodel_HyperDrive.pt')
#model = best_run.register_model(model_name='sklearn-iris', 
#                           model_path='outputs/model.joblib',
#                           model_framework=Model.Framework.SCIKITLEARN,
#                           model_framework_version='0.19.1',
#                           resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5))
joblib.dump(model, 'model_HyperDrive.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### ARIK ERTUGRUL ###
url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
dataset = TabularDatasetFactory.from_delimited_files(path=url)

In [ ]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(dataset)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

x_train['y']=y_train

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train
# https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-auto-train-models
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='regression',
    primary_metric='Accuracy',
    training_data=x_train,
    label_column_name='y',
    n_cross_validations=5)

In [2]:
# Submit your automl run

### ARIK ERTUGRUL ###
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train
experiment = Experiment(ws, "automl_test_experiment")
automl_run = experiment.submit(config=automl_config, show_output=True)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

In [ ]:
# Retrieve and save your best automl model.

### ARIK ERTUGRUL ###
best_run, fitted_model = automl_run.get_output()
print(best_run)
print(fitted_model)

y_predict = fitted_model.predict(x_test)
print(y_predict[:10])

best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print('\n learning rate:',parameter_values[3])
print('\n keep probability:',parameter_values[5])
print('\n batch size:',parameter_values[7])

model = best_run.register_model(model_name = 'bestmodel_automl', model_path = 'bestmodel_automl.pt')
joblib.dump(model, 'model_automl.joblib')
